# - Data Warehouse -

### Het samenvoegen van bestaande tabellen voor het uiteindelijke 'Datawarehouse'

##### Importeren van benodigde dependencies

In [ ]:
import pandas as pd
import pyodbc

import warnings
warnings.filterwarnings("ignore")

# Importeren van de create_connection en run_query functies uit de database_utils.py file
from utils.database_utils import create_connection, run_query

##### 1 - Opbouwen van dataframes voor elke database-tabel

In [ ]:

aw_PersonDF= run_query("SELECT * FROM Person.person", "AdventureWorks2019")
aw_EmailAdressDF= run_query("SELECT * FROM Person.EmailAddress", "AdventureWorks2019")
aw_PasswordDF= run_query("SELECT * FROM Person.Password", "AdventureWorks2019")

print(aw_PersonDF.columns)
print(aw_EmailAdressDF.columns)
print(aw_PasswordDF.columns)



Je kunt vergelijkbare queries uitvoeren voor andere tabellen die je wilt opnemen.

#### 2 - Data transformatie

Na het opbouwen van de dataframes voor elke database-tabel, kunnen we beginnen met het transformeren van de data. Dit omvat het samenvoegen van tabellen, het toepassen van filters, het uitvoeren van berekeningen, enzovoort.

In [ ]:
# Merge aw_PersonDF and aw_EmailAdressDF
aw_PersonDF.drop(columns=['rowguid', 'ModifiedDate'], inplace=True)
aw_EmailAdressDF.drop(columns=['rowguid', 'ModifiedDate'], inplace=True)
aw_PasswordDF.drop(columns=['rowguid', 'ModifiedDate'], inplace=True)


merged_df = pd.merge(aw_PersonDF, aw_EmailAdressDF, on='BusinessEntityID')
merged_df = pd.merge(merged_df, aw_PasswordDF, on='BusinessEntityID')

filtered_df = merged_df
filtered_df


In [ ]:
create_table_query = """

CREATE TABLE PersonInfo (
    BusinessEntityID int PRIMARY KEY,
    PersonType nchar(5),
    NameStyle bit,
    Title nvarchar(max),
    FirstName nvarchar(max),
    MiddleName nvarchar(max),
    LastName nvarchar(max),
    Suffix nvarchar(max),
    EmailPromotion int,
    AdditionalContactInfo xml,
    Demographics xml,
    EmailAddressID int,
    EmailAddress nvarchar(max),
    PasswordHash varchar(max),
    PasswordSalt varchar(max)
);
"""

# Create the table in SQL Server
db_name = "testDB"
conn, cursor = create_connection(db_name)
cursor.execute(create_table_query)
conn.commit()
conn.close()

#### 3 - Data loading

Na het transformeren van de data, kunnen we de resulterende dataframe in de doeltabel van ons datawarehouse laden.

In [ ]:
# Verbinding maken met de database
test_database_name = 'testDB'
conn, cursor = create_connection(test_database_name)

for index, row in filtered_df.iterrows():
    NameStyle = 1 if row['NameStyle'] else 0

    # Replace single quotes in the data
    title = row['Title'].replace("'", "''") if row['Title'] else None
    first_name = row['FirstName'].replace("'", "''") if row['FirstName'] else None
    middle_name = row['MiddleName'].replace("'", "''") if row['MiddleName'] else None
    last_name = row['LastName'].replace("'", "''") if row['LastName'] else None
    suffix = row['Suffix'].replace("'", "''") if row['Suffix'] else None
    additional_contact_info = row['AdditionalContactInfo'].replace("'", "''") if row['AdditionalContactInfo'] else None
    demographics = row['Demographics'].replace("'", "''") if row['Demographics'] else None
    email_address = row['EmailAddress'].replace("'", "''") if row['EmailAddress'] else None
    password_hash = row['PasswordHash'].replace("'", "''") if row['PasswordHash'] else None
    password_salt = row['PasswordSalt'].replace("'", "''") if row['PasswordSalt'] else None

    # Opstellen van de SQL-invoegquery
    query = f"""
        INSERT INTO PersonInfo (
        BusinessEntityID, PersonType, NameStyle, Title, FirstName,
        MiddleName, LastName, Suffix, EmailPromotion,
        AdditionalContactInfo, Demographics, EmailAddressID,
        EmailAddress, PasswordHash, PasswordSalt
        ) 
        VALUES (
        {row['BusinessEntityID']}, '{row['PersonType']}', {NameStyle}, '{title}', '{first_name}',
        '{middle_name}', '{last_name}', '{suffix}', {row['EmailPromotion']},
        '{additional_contact_info}', '{demographics}', {row['EmailAddressID']},
        '{email_address}', '{password_hash}', '{password_salt}'
        )
    """
    # Uitvoeren van de query
    cursor.execute(query)

conn.commit()
conn.close()

**Note:** Voeg indien nodig zoveel Markdown- of codeblokken toe als nodig is.

#### 4 -  Data Quality Checks

Voeg controles toe om de kwaliteit van de gegevens te waarborgen voordat ze worden geladen in het datawarehouse:

In [ ]:
# Controleren op ontbrekende waarden
missing_values = filtered_df.isnull().sum()

# Controleren op duplicaten
duplicate_rows = filtered_df.duplicated().sum()

# Weergave van resultaten
print("Aantal ontbrekende waarden:", missing_values)
print("Aantal duplicaten:", duplicate_rows)

**Note:** Dit is optioneel, het leek mij opzich best handig om te doen.